In [1]:
%matplotlib widget
from ipywidgets import *
import os
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from PIL import ImageOps
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter
from scipy.special import erf

In [2]:
## Define functions

def Sobel(im, k, N):
    """ Take the sobel of the image (x and y) with a kerner of k, normalized by N """
    (nrows, ncols) = im.shape
    sobelx = cv.Sobel(im,ddepth=cv.CV_64F, dx=1,dy=0,ksize=k) #cv.CV_64F
    sobely = cv.Sobel(im,ddepth=cv.CV_64F, dx=0,dy=1,ksize=k)
    sobel = np.zeros(im.shape)
    if N=='self':
        sobel = np.sqrt(sobelx**2 + sobely**2)
        sobel = sobel/np.amax(sobel)
    else: 
        sobel = np.sqrt(sobelx**2 + sobely**2)/N
    return sobel

def AddArtificialFeature(im, width_0_curved, start_0_curved):
    """ 
    Add artificial feature to normalize the sobel value (1.0 = max sharpness).
    White (2**12) square in the middle of a black (0) square. 
    'width_0_curved'-> width of each edge (total width = width_0_curved*3)
    'start_0_curved'-> bottom left corner (start position
    """
    im_withFeature = im
    width_256_curved = width_0_curved
    p0 = start_0_curved
    p1 = start_0_curved+width_0_curved
    p2 = start_0_curved+width_0_curved+width_256_curved
    p3 = start_0_curved+width_0_curved*2+width_256_curved
    im_withFeature[p0:p3,p0:p3] = 0.0 #np.amin(im)
    im_withFeature[p1:p2,p1:p2] = 2.0**12 #np.amax(im)
    return im_withFeature

def RemoveArtificialFeature(sobel, width_0_curved, start_0_curved, k):
    """ 
    Removes artificial feature added by 'AddArtificialFeature' from the image.
    Set the area to min image value.
    """
    sobel_removed = sobel
    width_256_curved = width_0_curved
    p0 = start_0_curved - k
    p3 = start_0_curved+width_0_curved*2+width_256_curved + k
    sobel_removed[p0:p3,p0:p3] = np.amin(sobel)
    return sobel_removed

def sobelModel(data,a,b,c,m,n,d): # data = (x,y)
    """ Function to fit the sobel simulation """
    x,y = data
    return (m*y+n)*(a*np.exp(-b*x)+c)+d

In [3]:
## Import image

current_directory = os.getcwd()
save_directory = '/Users/racicot.i/Documents/GitHub_CodeResults/SobelSharpness_Theory'
im_perfect_sub = np.load('/Users/racicot.i/repos/SobelSharpness_Theory/im_perfect_sub.npy') ## Might want to get a better one 


## Some parameters

## Artificial feature 
## --> Different for curved/flat detectors because of pixel size difference. (5.5*0.89/8.0)
width_0_curved = 22
width_256_curved = width_0_curved
start_0_curved = 25

width_0_flat = 17
width_256_flat = width_0_flat
start_0_flat = 25

# Sobel stuff

# threshold = 0.5

curved_cell_x, curved_cell_y = 220,220 
curved_sobel_k = 9   

flat_cell_x, flat_cell_y = 170,170                                              # --> 1.36mm
flat_sobel_k = 7

In [4]:
## Simulate blurring for different greyscale levels

k = 9 

# Define blurring params (sigma for Gaussian blurring)
blur_sigma_list = np.arange(0.0, 10.0, 0.05)
# Define greyscale params (max value)
greyscale_g_list = np.arange(2**12-1,int((2**12-1)/4),-20)

NNs = len(blur_sigma_list)
NNg = len(greyscale_g_list)
M_sub = np.amax(im_perfect_sub)
sobel_max_vals = np.zeros((NNg,NNs))
sobel_max_vals_fits = []

if k==9:
    width_0 = width_0_curved
    start_0 = start_0_curved
elif k==7:
    width_0 = width_0_flat
    start_0 = start_0_flat
else:
    print('Adust values of Artificial feature to the k value')
    

for i in range(0,NNs):
    for j in range(0,NNg):
        ## Get values of blurring (sigma) and greyscale (g)
        sigma = blur_sigma_list[i]
        g = greyscale_g_list[j]
        ## adjust greyscale
        im_perfect_g = im_perfect_sub/M_sub*g
        ## blurr  image
        im_blurred = gaussian_filter(im_perfect_g, sigma)
        ## add artificial feature that normalizes the sobel
        im_blurred_AF = AddArtificialFeature(im_blurred, width_0, start_0)
        ## take sobel
        im_blurred_sobel = Sobel(im_blurred_AF, k, 'self')
        ## remove artificial featre
        im_blurred_sobel_noAF = RemoveArtificialFeature(im_blurred_sobel, width_0, start_0, k)
        ## take value of max sobel
        M_blurred = np.round(np.amax(im_blurred_sobel_noAF),3)
        sobel_max_vals[j,i] = M_blurred
        ## fix to give the fit a chance because blurring with sigma<1.0 behaves oddly
        if i>=10: 
            sobel_max_vals_fits.append([sigma, g, M_blurred])
        
sobel_max_vals_fits = np.array(sobel_max_vals_fits)

## Save data
# np.save(save_directory+'/Data/SimMaxSobel_BlurringGreyscale_k'+str(k)+'.npy', 
#         sobel_max_vals_fits)

In [7]:
## Save results depending on the k value to compare them
if k==9:
    sobel_max_vals_fits_9 = sobel_max_vals_fits
    sobel_max_vals_9 = sobel_max_vals
if k==7:
    sobel_max_vals_fits_7 = sobel_max_vals_fits
    sobel_max_vals_7 = sobel_max_vals

In [8]:
## Plot results
fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(sobel_max_vals, cmap='jet',vmin=0.0, vmax=1.0) #,vmin=0.0, vmax=1.0
ax.set_title('Max sobel for different blurring and greyscales, k='+str(k))

# im = ax.imshow(np.divide(np.subtract(sobel_max_vals_9, sobel_max_vals_7),sobel_max_vals_9)*100, cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7/k9 %')

# im = ax.imshow(np.subtract(sobel_max_vals_9, sobel_max_vals_7), cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7')


ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [max value]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(0,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
# plt.savefig(save_directory+'/SimMaxSobel_BlurringGreyscale_k'+str(k)+'.png', transparent=True)
# plt.savefig(save_directory+'/Results/SimMaxSobel_diff_k9-k7.png', transparent=True)
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
## Fit the 2D surface

initial_guess = 1.49347127e+00, 4.02525068e-01, 1.93467577e-01, 1.70251464e-04, -1.20819172e-05, 7.50579437e-07 #a,b,c,m,n,d

xx = sobel_max_vals_fits[:,0]
yy = sobel_max_vals_fits[:,1]
zz = sobel_max_vals_fits[:,2]
popt, pcov = curve_fit(sobelModel, (xx,yy), zz, initial_guess)
# ## Errors on fit
perr = np.sqrt(np.diag(pcov))
print('k=',k)
print('fit parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt))
print('fit errors on parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr))
zz_fit = sobelModel((xx,yy),*popt).reshape((NNg,NNs-10),order='F')
zz_data = zz.reshape((NNg,NNs-10),order='F')

 #a,b,c,m,n,d
fit_popt_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt)
fit_perr_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr)

k= 9
fit parameters: a=1.469, b=0.403, c=0.190, m=0.000173, n=-0.000013, d=0.000
fit errors on parameters: a=9382.936, b=0.000, c=1215.511, m=1.105946, n=0.082013, d=0.000


In [ ]:
## Make figures

### ************************************************************************************************
## FIT

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit, cmap='jet',vmin=0.0, vmax=1.0)
ax.set_title('FIT: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale (max value)')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/SimMaxSobel_k'+str(k)+'_fit.png', 
            transparent=True)
plt.close()

### ************************************************************************************************
## DIFFERENCE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit-sobel_max_vals[:,0:-10], cmap='jet')
ax.set_title('DIFFERENCE fit - data, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale (max value)')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/SimMaxSobel_k'+str(k)+'_difference.png', 
            transparent=True)
plt.close()


### ************************************************************************************************
## DIFFERENCE PERCENTAGE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(np.divide((zz_fit-zz_data),zz_data)*100, cmap='jet')
ax.set_title('DIFFERENCE PERCENTAGE (fit - data)/data*100, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale (max value)')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/SimMaxSobel_k'+str(k)+'_differencePerc.png', 
            transparent=True)
plt.close()

### ************************************************************************************************
## DATA

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_data, cmap='jet',vmin=0.0, vmax=1.0)
ax.set_title('DATA: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale (max value)')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/SimMaxSobel_k'+str(k)+'_data.png', 
            transparent=True)
plt.close()


In [ ]:
## Different way to handle greyscale

In [ ]:
gg = 1000
ggg = 2**12 -gg

test = im_perfect_sub/np.amax(im_perfect_sub)*(2**12-ggg) +ggg

test[0,0] = 0.0
test[0,1] = 2**12
print(np.amin(test[1:,:]), np.amax(test))
plt.figure()
plt.imshow(test,cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
k = 9 

# Define blurring params (sigma for Gaussian blurring)
blur_sigma_list = np.arange(0.0, 10.0, 0.05)
# Define greyscale params (max value)
greyscale_g_list = np.arange(2**12-1,int((2**12-1)/4),-20)

NNs = len(blur_sigma_list)
NNg = len(greyscale_g_list)
M_sub = np.amax(im_perfect_sub)
sobel_max_vals_2 = np.zeros((NNg,NNs))
sobel_max_vals_fits_2 = []

if k==9:
    width_0 = width_0_curved
    start_0 = start_0_curved
elif k==7:
    width_0 = width_0_flat
    start_0 = start_0_flat
else:
    print('Adust values of Artificial feature to the k value')
    

for i in range(0,NNs):
    for j in range(0,NNg):
        ## Get values of blurring (sigma) and greyscale (g)
        sigma = blur_sigma_list[i]
        g = greyscale_g_list[j] # depth of the greyscale
#         print('g ', g)
        ## adjust greyscale, now raising the black instead of lowering the white
        gg = 2**12 - g
        im_perfect_g = im_perfect_sub/np.amax(im_perfect_sub)*(2**12-gg) +gg
#         print('im_perfect_g ', np.amin(im_perfect_g), np.amax(im_perfect_g))
        ## blurr  image
        im_blurred = gaussian_filter(im_perfect_g, sigma)
#         print('im_blurred ', np.amin(im_blurred), np.amax(im_blurred))
        ## add artificial feature that normalizes the sobel
        im_blurred_AF = AddArtificialFeature(im_blurred, width_0, start_0)
        
        ## take sobel
        im_blurred_sobel = Sobel(im_blurred_AF, k, 'self')
        ## remove artificial featre
        im_blurred_sobel_noAF = RemoveArtificialFeature(im_blurred_sobel, width_0, start_0, k)
#         print('im_blurred_sobel_noAF ', np.amin(im_blurred_sobel_noAF), np.amax(im_blurred_sobel_noAF))
        ## take value of max sobel
        M_blurred = np.round(np.amax(im_blurred_sobel_noAF),3)
#         print('M_blurred ', M_blurred)
        sobel_max_vals_2[j,i] = M_blurred
        ## fix to give the fit a chance because blurring with sigma<1.0 behaves oddly
        if i>=10: 
            sobel_max_vals_fits_2.append([sigma, g, M_blurred])
        
sobel_max_vals_fits_2 = np.array(sobel_max_vals_fits_2)

## Save data
np.save(save_directory+'/Data/NewGreyscale_k'+str(k)+'.npy', 
        sobel_max_vals_fits_2)

In [ ]:
## Plot results
fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(sobel_max_vals_2, cmap='jet',vmin=0.0, vmax=1.0) #,vmin=0.0, vmax=1.0
ax.set_title('Max sobel for different blurring and greyscales, k='+str(k))

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [max value]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(0,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/NewGreyscale_k'+str(k)+'.png', transparent=True)
plt.show()



In [ ]:
## Fit the 2D surface

initial_guess = 1.49347127e+00, 4.02525068e-01, 1.93467577e-01, 1.70251464e-04, -1.20819172e-05, 7.50579437e-07 #a,b,c,m,n,d

xx = sobel_max_vals_fits_2[:,0]
yy = sobel_max_vals_fits_2[:,1]
zz = sobel_max_vals_fits_2[:,2]
popt, pcov = curve_fit(sobelModel, (xx,yy), zz, initial_guess)
# ## Errors on fit
perr = np.sqrt(np.diag(pcov))
print('Alternative greyscale:')
print('fit parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt))
print('fit errors on parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr))
zz_fit = sobelModel((xx,yy),*popt).reshape((NNg,NNs-10),order='F')
zz_data = zz.reshape((NNg,NNs-10),order='F')

 #a,b,c,m,n,d
fit_popt_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt)
fit_perr_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr)

In [ ]:
## Plot difference
diff = np.subtract(sobel_max_vals, sobel_max_vals_2)

print('Subraction extremum: ',np.amin(diff), np.amax(diff))
fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(diff, cmap='jet') #,vmin=0.0, vmax=1.0
ax.set_title('Difference betweeen greyscales (lowering white - raising black), k='+str(k))

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [max value]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(0,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
Yticks_val.append(np.round(greyscale_g_list[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/NewGreyscale_comparison_k'+str(k)+'.png', transparent=True)
plt.show()



In [30]:
###
"""
Weber contrast 

M = (I_max - I_min)/(I_min)

"""
def MichelsonContrast(im):
    M = np.amax(im)
    m = np.amin(im)
    val = (M-m)/(m+M)
    return val

def WeberContrast(im):
    M = np.amax(im)
#     m = np.amin(im)
    m = 0.1
    val = (M-m)/(m)
    return val

def Range(im):
    M = np.amax(im)
    m = np.amin(im)
    val = M-m
    return val


In [31]:
## Simulate blurring for different greyscale levels

k = 9 

# Define blurring params (sigma for Gaussian blurring)
blur_sigma_list = np.arange(0.0, 10.0, 0.05)
# Define greyscale params (max value)
greyscale_g_list = np.arange(2**12-1,int((2**12-1)/4),-20)

NNs = len(blur_sigma_list)
NNg = len(greyscale_g_list)
M_sub = np.amax(im_perfect_sub)
sobel_max_vals_3 = np.zeros((NNg,NNs))
sobel_max_vals_fits_3 = []

if k==9:
    width_0 = width_0_curved
    start_0 = start_0_curved
elif k==7:
    width_0 = width_0_flat
    start_0 = start_0_flat
else:
    print('Adust values of Artificial feature to the k value')
    

for i in range(0,NNs):
    for j in range(0,NNg):
        ## Get values of blurring (sigma) and greyscale (g)
        sigma = blur_sigma_list[i]
        g = greyscale_g_list[j]
        gg = 2**12 - g
        
        ## adjust greyscale
        
#         im_perfect_g = im_perfect_sub/M_sub*g
        im_perfect_g = im_perfect_sub/np.amax(im_perfect_sub)*(2**12-gg) +gg
    
        ## blurr  image
        im_blurred = gaussian_filter(im_perfect_g, sigma)
        ## add artificial feature that normalizes the sobel
        im_blurred_AF = AddArtificialFeature(im_blurred, width_0, start_0)
        ## take sobel
        im_blurred_sobel = Sobel(im_blurred_AF, k, 'self')
        ## remove artificial featre
        im_blurred_sobel_noAF = RemoveArtificialFeature(im_blurred_sobel, width_0, start_0, k)
        ## take value of max sobel
        M_blurred = np.round(np.amax(im_blurred_sobel_noAF),3)
        ## Michelson contrast
#         Michelson = MichelsonContrast(im_perfect_g)
        ## Range
        range_val = Range(im_perfect_g)
        ## Weber contrast
#         Weber = WeberContrast(im_perfect_g)
        ## fix to give the fit a chance because blurring with sigma<1.0 behaves oddly
        if i>=10: 
#             sobel_max_vals_fits_3.append([sigma, g, Michelson, M_blurred])
            sobel_max_vals_fits_3.append([sigma, g, range_val, M_blurred])
#             sobel_max_vals_fits_3.append([sigma, g, Weber, M_blurred])
        
sobel_max_vals_fits_3 = np.array(sobel_max_vals_fits_3)

## Save data
# np.save(save_directory+'/Data/MichelsonContrast_k'+str(k)+'.npy', 
#         sobel_max_vals_fits_3)

In [32]:
Coeffs = sobel_max_vals_fits_3[:,2]
Coeffs = Coeffs.reshape((NNg,NNs-10),order='F')

GG = sobel_max_vals_fits_3[:,1]
GG = GG.reshape((NNg,NNs-10),order='F')

In [34]:
plt.figure()
plt.plot(GG[:,0], Coeffs[:,0])
plt.xlabel('Max white value')
plt.ylabel('Range value')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
## Fit the 2D surface

# initial_guess = 1.49347127e+00, 4.02525068e-01, 1.93467577e-01, 1.70251464e-04, -1.20819172e-05, 7.50579437e-07 #a,b,c,m,n,d

## [sigma, g, Michelson, M_blurred]
xx = sobel_max_vals_fits_3[:,0] ## sigma
yy = sobel_max_vals_fits_3[:,2] ## Michelson contrast
zz = sobel_max_vals_fits_3[:,3] ## sobel blurred image
popt, pcov = curve_fit(sobelModel, (xx,yy), zz)
# ## Errors on fit
perr = np.sqrt(np.diag(pcov))
# print('Alternative greyscale, Michelson contrast:')
print('Alternative greyscale, Range value:')
print('k=',k)
print('fit parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt))
print('fit errors on parameters: a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr))
zz_fit = sobelModel((xx,yy),*popt).reshape((NNg,NNs-10),order='F')
zz_data = zz.reshape((NNg,NNs-10),order='F')

 #a,b,c,m,n,d
fit_popt_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(popt)
fit_perr_str = 'a=%5.3f, b=%5.3f, c=%5.3f, m=%5.6f, n=%5.6f, d=%5.3f' % tuple(perr)

Alternative greyscale, Range value:
k= 9
fit parameters: a=11.303, b=0.403, c=1.464, m=0.000022, n=-0.000002, d=0.000
fit errors on parameters: a=106411.144, b=0.000, c=13785.090, m=0.211787, n=0.015071, d=0.000


In [29]:
## Plot results


### ************************************************************************************************
## DATA


fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_data, cmap='jet',vmin=0.0, vmax=1.0) #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k='+str(k))
ax.set_title('DATA: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

# im = ax.imshow(np.divide(np.subtract(sobel_max_vals_9, sobel_max_vals_7),sobel_max_vals_9)*100, cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7/k9 %')

# im = ax.imshow(np.subtract(sobel_max_vals_9, sobel_max_vals_7), cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7')


ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Michelson contrast]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))


stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DATA_MichelsonContrast_'+str(k)+'.png', transparent=True)
plt.show()

### ************************************************************************************************
## FIT


fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit, cmap='jet',vmin=0.0, vmax=1.0)
ax.set_title('FIT: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Michelson contrast]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/FIT_MichelsonContrast_'+str(k)+'.png', 
            transparent=True)
plt.close()

### ************************************************************************************************
## DIFFERENCE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit-zz_data, cmap='jet')
ax.set_title('DIFFERENCE fit - data, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Michelson contrast]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DIFF_MichelsonContrast_'+str(k)+'.png', 
            transparent=True)
plt.close()


### ************************************************************************************************
## DIFFERENCE PERCENTAGE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(np.divide((zz_fit-zz_data),zz_data)*100, cmap='jet')
ax.set_title('DIFFERENCE PERCENTAGE (fit - data)/data*100, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Michelson contrast]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DIFF-PERC_MichelsonContrast_k'+str(k)+'.png', 
            transparent=True)
plt.close()

### ************************************************************************************************


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
## Plot results


### ************************************************************************************************
## DATA


fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_data, cmap='jet',vmin=0.0, vmax=1.0) #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k='+str(k))
ax.set_title('DATA: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

# im = ax.imshow(np.divide(np.subtract(sobel_max_vals_9, sobel_max_vals_7),sobel_max_vals_9)*100, cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7/k9 %')

# im = ax.imshow(np.subtract(sobel_max_vals_9, sobel_max_vals_7), cmap='jet') #,vmin=0.0, vmax=1.0
# ax.set_title('Max sobel for different blurring and greyscales, k9-k7')


ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Range]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))


stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DATA_Range_'+str(k)+'.png', transparent=True)
plt.show()

### ************************************************************************************************
## FIT


fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit, cmap='jet',vmin=0.0, vmax=1.0)
ax.set_title('FIT: max sobel for different blurring and greyscales, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Range]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/FIT_Range_'+str(k)+'.png', 
            transparent=True)
plt.close()

### ************************************************************************************************
## DIFFERENCE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(zz_fit-zz_data, cmap='jet')
ax.set_title('DIFFERENCE fit - data, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Range]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
# Yticks_val = [greyscale_g_list[i*stepy] for i in range(0,Nticks)]
# Yticks_val.append(np.round(greyscale_g_list[-1],1))
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DIFF_Range_'+str(k)+'.png', 
            transparent=True)
plt.close()


### ************************************************************************************************
## DIFFERENCE PERCENTAGE

fig = plt.figure(figsize=(11,7))
ax = fig.add_subplot(1, 1, 1)

im = ax.imshow(np.divide((zz_fit-zz_data),zz_data)*100, cmap='jet')
ax.set_title('DIFFERENCE PERCENTAGE (fit - data)/data*100, (m*y+n)*(a*np.exp(-b*x)+c)+d \n'+
             'fit params: '+fit_popt_str)#+'\n fit errors: '+fit_perr_str)

ax.set_xlabel('Blurring [sigma]')
ax.set_ylabel('Greyscale [Range]')
## ticks
Nticks = 10

stepx = int(np.round(NNs/(Nticks),0))
Xticks_loc = [i*1.0/Nticks*NNs for i in range(0,Nticks)]
# Xticks_loc.append(NNs)
Xticks_val = [blur_sigma_list[i*stepx] for i in range(1,Nticks)]
Xticks_val.append(np.round(blur_sigma_list[-1],1))

stepy = int(np.round(NNg/(Nticks),0))
Yticks_loc = [i*1.0/Nticks*NNg for i in range(0,Nticks)]
Yticks_loc.append(NNg)
Yticks_val = [np.round(yy[i*stepy],1) for i in range(0,Nticks)]
Yticks_val.append(np.round(yy[-1],1))

ax.set_xticks(Xticks_loc)
ax.set_xticklabels(Xticks_val)
ax.set_yticks(Yticks_loc)
ax.set_yticklabels(Yticks_val)

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

plt.tight_layout()
plt.savefig(save_directory+'/DIFF-PERC_Range_k'+str(k)+'.png', 
            transparent=True)
plt.close()

### ************************************************************************************************


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …